In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import hdbscan
from sklearn.manifold import TSNE

In [179]:
cluster_df = pd.read_csv('results/clustering_result2.csv', encoding='utf-8-sig', index_col=0)
tag_top = pd.read_csv('../../datas/top5_tags_0720.csv', encoding='utf-8-sig', index_col=0)
api_data = pd.read_csv('../../datas/camp_api_info_210619.csv', encoding='utf-8-sig', index_col=0)
eda_camp = pd.merge(cluster_df, tag_top, how='left', left_on='contentId', right_on='index')

/Users/hyunsil/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [2]:
# eda_camp2 = pd.merge(eda_camp, api_data[['contentId', 'mapX', 'mapY','sigungucode']])

In [3]:
# plt.figure(figsize=(14,10))
# ax = sns.scatterplot(data=eda_camp2, x='mapX', y='mapY', hue='cluster')
# plt.setp(ax.get_legend().get_texts(), fontsize='10')
# plt.show()

In [4]:
# si = []
# for i in range(len(eda_camp2)):
#     si.append(str(eda_camp2['sigungucode'][i])[:2])
# eda_camp2['si']=si

In [5]:
# cluster_number = eda_camp2[eda_camp2['cluster'] == 0]

# plt.figure(figsize=(14,10))
# ax = sns.scatterplot(data=cluster_number, x='mapX', y='mapY', hue='si')
# plt.setp(ax.get_legend().get_texts(), fontsize='10')
# plt.show()

In [268]:
import tomotopy as tp
import pyLDAvis

def cluster_lda2(number):
    cluster_number = eda_camp[eda_camp['cluster'] == number]
    cluster_number = cluster_number.reset_index(drop=True)
    cluster_number = cluster_number[cluster_number['top5_tags'] != '[]']
    cluster_number['top5_tags'] = cluster_number['top5_tags'].str.replace('[', '').str.replace(']','').str.replace("'", '')
    
    model = tp.DMRModel(k=5, rm_top=0, alpha=0.1, eta=0.01, tw=tp.TermWeight.IDF)
    model.set_word_prior('가족', [1.0 if k == 0 else 0.1 for k in range(model.k)])
    model.set_word_prior('시설이 좋은', [1.0 if k == 1 else 0.1 for k in range(model.k)])
    model.set_word_prior('친절한', [1.0 if k == 2 else 0.1 for k in range(model.k)])
    

    result = []
    for line in cluster_number['top5_tags']:
        model.add_doc(line.strip().split(', '))

    for i in range(100):
        model.train(i)

    for i in range(model.k):
        #print('Top 10 words of topic {}'.format(k))
        model.get_topic_words(i, top_n=30)
        result.append(model.get_topic_words(i, top_n=30))
    
    topic_term_dists = np.stack([model.get_topic_word_dist(k) for k in range(model.k)])
    doc_topic_dists = np.stack([doc.get_topic_dist() for doc in model.docs])
    doc_topic_dists /= doc_topic_dists.sum(axis=1, keepdims=True)
    doc_lengths = np.array([len(doc.words) for doc in model.docs])
    vocab = list(model.used_vocabs)
    term_frequency = model.used_vocab_freq

    prepared_data = pyLDAvis.prepare(
        topic_term_dists, 
        doc_topic_dists, 
        doc_lengths, 
        vocab, 
        term_frequency,
        start_index=0, # tomotopy starts topic ids with 0, pyLDAvis with 1
        sort_topics=False # IMPORTANT: otherwise the topic_ids between pyLDAvis and tomotopy are not matching!
    )
    pyLDAvis.save_html(prepared_data, f'./results/ldavis{number}.html')


    
    df = pd.DataFrame(result[0])
    df2 = pd.DataFrame(result[1])
    df3 = pd.DataFrame(result[2])
    df4 = pd.DataFrame(result[3])
    df5 = pd.DataFrame(result[4])
    re_df = pd.concat([df,df2,df3,df4,df5],1)
    
    return re_df

/Users/hyunsil/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [269]:
cluster_lda2(0) 

/Users/hyunsil/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,0,1,0,1,0,1,0,1
0,차 대기 편한,0.108537,시설이 좋은,0.598584,축제,0.403229,시설이 좋은,0.328642,친절한,0.440539
1,사이트 간격이 넓은,0.105400,친절한,0.232523,생태교육,0.288044,접근성 좋은,0.323204,온수 잘 나오는,0.362757
2,아이들 놀기 좋은,0.105354,사이트 간격이 넓은,0.129724,문화유적,0.250646,수영장 있는,0.170339,가족,0.169141
3,온수 잘 나오는,0.105321,가족,0.030150,가족,0.050447,친절한,0.141087,시설이 좋은,0.001969
4,커플,0.102970,접근성 좋은,0.000694,사이트 간격이 넓은,0.000587,커플,0.018933,사이트 간격이 넓은,0.001969
5,가족,0.073883,둘레길,0.000694,재미있는,0.000587,가족,0.017162,접근성 좋은,0.001969
6,물놀이 하기 좋은,0.072949,아이들 놀기 좋은,0.000694,둘레길,0.000587,사이트 간격이 넓은,0.000058,둘레길,0.001969
7,문화유적,0.067991,커플,0.000694,아이들 놀기 좋은,0.000587,아이들 놀기 좋은,0.000058,아이들 놀기 좋은,0.001969
8,생태교육,0.054715,차 대기 편한,0.000694,접근성 좋은,0.000587,재미있는,0.000058,차 대기 편한,0.001969
9,둘레길,0.049573,문화유적,0.000694,커플,0.000587,둘레길,0.000058,문화유적,0.001969


In [270]:
cluster_lda2(1)

/Users/hyunsil/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,0,1,0,1,0,1,0,1
0,온수 잘 나오는,0.077350,시설이 좋은,0.227954,먹거리가 풍부한,0.367647,시설이 좋은,0.716884,축제,0.181318
1,사이트 간격이 넓은,0.077036,접근성 좋은,0.154203,접근성 좋은,0.233778,가족,0.239497,생태교육,0.169986
2,아이들 놀기 좋은,0.076905,힐링,0.121569,뷰가 좋은,0.193822,사이트 간격이 넓은,0.001678,문화유적,0.156094
3,차 대기 편한,0.076441,뷰가 좋은,0.115048,시설이 좋은,0.127426,생태교육,0.001678,재미있는,0.123531
4,커플,0.068764,친절한,0.105021,친절한,0.073207,힐링,0.001678,수영장 있는,0.111201
5,여유있는,0.068761,수영장 있는,0.076843,힐링,0.000179,바다가 보이는,0.001678,물놀이 하기 좋은,0.094730
6,가족,0.067955,먹거리가 풍부한,0.055985,그늘이 많은,0.000179,수영장 있는,0.001678,둘레길,0.049118
7,계곡 옆,0.064859,계곡 옆,0.034201,바다가 보이는,0.000179,그늘이 많은,0.001678,가족,0.021906
8,별보기 좋은,0.054058,커플,0.026576,생태교육,0.000179,물놀이 하기 좋은,0.001678,커플,0.020901
9,그늘이 많은,0.051143,가족,0.026307,물놀이 하기 좋은,0.000179,커플,0.001678,사이트 간격이 넓은,0.018873


In [271]:
cluster_lda2(2) 

/Users/hyunsil/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,0,1,0,1,0,1,0,1
0,문화유적,0.137103,시설이 좋은,0.220683,축제,0.235641,물맑은,0.190656,온수 잘 나오는,0.086690
1,수영장 있는,0.129699,접근성 좋은,0.173124,생태교육,0.206590,별보기 좋은,0.170153,차 대기 편한,0.081392
2,둘레길,0.126667,깨끗한,0.138432,문화유적,0.192915,그늘이 많은,0.163413,커플,0.080623
3,축제,0.123692,뷰가 좋은,0.136854,아이들 놀기 좋은,0.120537,여유있는,0.149963,아이들 놀기 좋은,0.077852
4,생태교육,0.121420,친절한,0.098080,커플,0.111927,계곡 옆,0.117271,사이트 간격이 넓은,0.077655
5,재미있는,0.117517,먹거리가 풍부한,0.070865,가족,0.089043,힐링,0.097633,가족,0.077007
6,물놀이 하기 좋은,0.111908,힐링,0.070378,친절한,0.039370,바다가 보이는,0.067714,친절한,0.061416
7,자전거 타기 좋은,0.087728,수영장 있는,0.049161,재미있는,0.000189,사이트 간격이 넓은,0.023642,계곡 옆,0.048341
8,사이트 간격이 넓은,0.012614,계곡 옆,0.022598,힐링,0.000189,생태교육,0.007078,물놀이 하기 좋은,0.045486
9,익스트림,0.009762,가족,0.019524,여유있는,0.000189,가족,0.006097,힐링,0.044232


In [272]:
cluster_lda2(3) 

/Users/hyunsil/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,0,1,0,1,0,1,0,1
0,가족,0.817311,접근성 좋은,0.296060,접근성 좋은,0.178158,자전거 타기 좋은,0.421656,별보기 좋은,0.070085
1,재미있는,0.007028,시설이 좋은,0.254585,친절한,0.148804,차 대기 편한,0.343573,아이들 놀기 좋은,0.070002
2,커플,0.007027,뷰가 좋은,0.184274,수영장 있는,0.144273,시설이 좋은,0.107062,계곡 옆,0.069850
3,그늘이 많은,0.007027,친절한,0.134054,뷰가 좋은,0.136432,힐링,0.079187,여유있는,0.069535
4,여유있는,0.007027,힐링,0.104645,시설이 좋은,0.129632,친절한,0.042416,커플,0.069039
5,친절한,0.007027,커플,0.021754,먹거리가 풍부한,0.109433,커플,0.000278,그늘이 많은,0.064933
6,힐링,0.007027,재미있는,0.000220,깨끗한,0.088211,깨끗한,0.000278,물맑은,0.064445
7,물놀이 하기 좋은,0.007027,둘레길,0.000220,힐링,0.063893,물놀이 하기 좋은,0.000278,가족,0.060497
8,깨끗한,0.007027,깨끗한,0.000220,커플,0.000061,뷰가 좋은,0.000278,힐링,0.050730
9,뷰가 좋은,0.007027,여유있는,0.000220,둘레길,0.000061,물맑은,0.000278,온수 잘 나오는,0.048053


In [273]:
cluster_lda2(4)

/Users/hyunsil/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,0,1,0,1,0,1,0,1
0,아이들 놀기 좋은,0.090564,깨끗한,0.144975,시설이 좋은,0.436411,축제,0.208872,물맑은,0.196718
1,커플,0.085888,시설이 좋은,0.143424,친절한,0.330758,문화유적,0.203620,별보기 좋은,0.194805
2,사이트 간격이 넓은,0.074739,뷰가 좋은,0.132870,가족,0.212670,생태교육,0.151321,여유있는,0.183633
3,온수 잘 나오는,0.074100,접근성 좋은,0.132870,깨끗한,0.000840,재미있는,0.123632,그늘이 많은,0.167184
4,가족,0.069239,힐링,0.102425,계곡 옆,0.000840,물놀이 하기 좋은,0.117026,계곡 옆,0.140317
5,여유있는,0.064756,친절한,0.100582,아이들 놀기 좋은,0.000840,둘레길,0.114275,힐링,0.084222
6,차 대기 편한,0.060808,수영장 있는,0.084550,커플,0.000840,수영장 있는,0.079276,가족,0.023841
7,계곡 옆,0.056009,먹거리가 풍부한,0.081121,물놀이 하기 좋은,0.000840,커플,0.000099,친절한,0.007491
8,별보기 좋은,0.051331,계곡 옆,0.022635,생태교육,0.000840,별보기 좋은,0.000099,커플,0.000094
9,그늘이 많은,0.048388,가족,0.021171,접근성 좋은,0.000840,아이들 놀기 좋은,0.000099,아이들 놀기 좋은,0.000094


In [274]:
cluster_lda2(5) 

/Users/hyunsil/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,0,1,0,1,0,1,0,1
0,계곡 옆,0.165393,접근성 좋은,0.355660,아이들 놀기 좋은,0.089989,가족,0.579486,뷰가 좋은,0.200404
1,별보기 좋은,0.149066,시설이 좋은,0.315515,커플,0.086417,뷰가 좋은,0.016176,접근성 좋은,0.191055
2,여유있는,0.146375,친절한,0.144136,온수 잘 나오는,0.086104,계곡 옆,0.016176,깨끗한,0.165393
3,그늘이 많은,0.143585,뷰가 좋은,0.080181,사이트 간격이 넓은,0.085648,아이들 놀기 좋은,0.016175,시설이 좋은,0.146888
4,힐링,0.125613,커플,0.076459,차 대기 편한,0.081763,먹거리가 풍부한,0.016174,힐링,0.116075
5,물맑은,0.113252,가족,0.021624,물놀이 하기 좋은,0.079696,차 대기 편한,0.016174,먹거리가 풍부한,0.101726
6,바다가 보이는,0.080852,아이들 놀기 좋은,0.000306,문화유적,0.069668,커플,0.016174,친절한,0.077362
7,시설이 좋은,0.044199,차 대기 편한,0.000306,친절한,0.065479,온수 잘 나오는,0.016174,자전거 타기 좋은,0.000055
8,가족,0.028034,온수 잘 나오는,0.000306,생태교육,0.062224,친절한,0.016174,아이들 놀기 좋은,0.000055
9,친절한,0.003176,문화유적,0.000306,둘레길,0.062224,문화유적,0.016174,차 대기 편한,0.000055


In [ ]:
##LDA모델

In [329]:
import tomotopy as tp
import pyLDAvis

def cluster_lda2(number):
    cluster_number = eda_camp[eda_camp['cluster'] == number]
    cluster_number = cluster_number.reset_index(drop=True)
    cluster_number = cluster_number[cluster_number['top5_tags'] != '[]']
    cluster_number['top5_tags'] = cluster_number['top5_tags'].str.replace('[', '').str.replace(']','').str.replace("'", '')
    
    model = tp.LDAModel(k=5, rm_top=0, alpha=0.1, eta=0.01, tw=tp.TermWeight.IDF, seed=6)
    model.set_word_prior('가족', [1.0 if k == 0 else 0.1 for k in range(model.k)])
    model.set_word_prior('시설이 좋은', [1.0 if k == 1 else 0.1 for k in range(model.k)])
    model.set_word_prior('친절한', [1.0 if k == 2 else 0.1 for k in range(model.k)])
    

    result = []
    for line in cluster_number['top5_tags']:
        model.add_doc(line.strip().split(', '))

    for i in range(200):
        model.train(i)

    for i in range(model.k):
        #print('Top 10 words of topic {}'.format(k))
        model.get_topic_words(i, top_n=30)
        result.append(model.get_topic_words(i, top_n=30))
    
    topic_term_dists = np.stack([model.get_topic_word_dist(k) for k in range(model.k)])
    doc_topic_dists = np.stack([doc.get_topic_dist() for doc in model.docs])
    doc_topic_dists /= doc_topic_dists.sum(axis=1, keepdims=True)
    doc_lengths = np.array([len(doc.words) for doc in model.docs])
    vocab = list(model.used_vocabs)
    term_frequency = model.used_vocab_freq

    prepared_data = pyLDAvis.prepare(
        topic_term_dists, 
        doc_topic_dists, 
        doc_lengths, 
        vocab, 
        term_frequency,
        start_index=0, # tomotopy starts topic ids with 0, pyLDAvis with 1
        sort_topics=False # IMPORTANT: otherwise the topic_ids between pyLDAvis and tomotopy are not matching!
    )
    pyLDAvis.save_html(prepared_data, f'./results2/ldavis{number}.html')


    
    df = pd.DataFrame(result[0])
    df2 = pd.DataFrame(result[1])
    df3 = pd.DataFrame(result[2])
    df4 = pd.DataFrame(result[3])
    df5 = pd.DataFrame(result[4])
    re_df = pd.concat([df,df2,df3,df4,df5],1)
    
    return re_df

/Users/hyunsil/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [330]:
cluster_lda2(0) 

/Users/hyunsil/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,0,1,0,1,0,1,0,1
0,가족,0.941116,시설이 좋은,0.701556,접근성 좋은,0.341303,차 대기 편한,0.103038,접근성 좋은,0.444609
1,아이들 놀기 좋은,0.003682,친절한,0.293795,수영장 있는,0.306503,사이트 간격이 넓은,0.102878,시설이 좋은,0.387908
2,온수 잘 나오는,0.003681,사이트 간격이 넓은,0.000310,시설이 좋은,0.241818,온수 잘 나오는,0.102762,친절한,0.148635
3,재미있는,0.003681,접근성 좋은,0.000310,친절한,0.108432,커플,0.102723,가족,0.017115
4,시설이 좋은,0.003680,둘레길,0.000310,사이트 간격이 넓은,0.000150,아이들 놀기 좋은,0.100016,사이트 간격이 넓은,0.000133
5,커플,0.003680,재미있는,0.000310,재미있는,0.000150,가족,0.072070,아이들 놀기 좋은,0.000133
6,둘레길,0.003680,아이들 놀기 좋은,0.000310,둘레길,0.000150,문화유적,0.071000,커플,0.000133
7,사이트 간격이 넓은,0.003680,커플,0.000310,아이들 놀기 좋은,0.000150,물놀이 하기 좋은,0.069254,온수 잘 나오는,0.000133
8,접근성 좋은,0.003680,가족,0.000310,가족,0.000150,생태교육,0.059361,차 대기 편한,0.000133
9,문화유적,0.003680,문화유적,0.000310,문화유적,0.000150,친절한,0.047274,문화유적,0.000133


In [331]:
cluster_lda2(1) 

/Users/hyunsil/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,0,1,0,1,0,1,0,1
0,문화유적,0.181767,먹거리가 풍부한,0.297396,시설이 좋은,0.270125,별보기 좋은,0.129379,온수 잘 나오는,0.098551
1,생태교육,0.179951,접근성 좋은,0.171066,접근성 좋은,0.179966,계곡 옆,0.114382,아이들 놀기 좋은,0.095597
2,축제,0.140356,뷰가 좋은,0.167930,힐링,0.127046,여유있는,0.108709,차 대기 편한,0.094986
3,수영장 있는,0.139518,시설이 좋은,0.153023,뷰가 좋은,0.116038,그늘이 많은,0.105807,사이트 간격이 넓은,0.094128
4,재미있는,0.126812,힐링,0.084417,친절한,0.115028,물맑은,0.088499,가족,0.076337
5,물놀이 하기 좋은,0.116269,친절한,0.081085,수영장 있는,0.085053,힐링,0.076756,커플,0.075039
6,둘레길,0.077028,수영장 있는,0.044214,계곡 옆,0.038439,커플,0.059958,친절한,0.058558
7,가족,0.019334,생태교육,0.000041,가족,0.027178,가족,0.051139,문화유적,0.057187
8,커플,0.018448,바다가 보이는,0.000041,깨끗한,0.017708,바다가 보이는,0.050592,물놀이 하기 좋은,0.048747
9,사이트 간격이 넓은,0.000027,그늘이 많은,0.000041,커플,0.016601,사이트 간격이 넓은,0.042461,생태교육,0.043116


In [332]:
cluster_lda2(2) 

/Users/hyunsil/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,0,1,0,1,0,1,0,1
0,별보기 좋은,0.175713,시설이 좋은,0.216442,둘레길,0.176849,생태교육,0.191215,온수 잘 나오는,0.092306
1,물맑은,0.171248,접근성 좋은,0.180211,축제,0.149878,문화유적,0.178549,차 대기 편한,0.086664
2,그늘이 많은,0.162151,깨끗한,0.145018,문화유적,0.128742,수영장 있는,0.176672,아이들 놀기 좋은,0.086083
3,여유있는,0.150063,뷰가 좋은,0.130334,자전거 타기 좋은,0.119984,재미있는,0.148974,커플,0.084365
4,계곡 옆,0.126097,친절한,0.099960,생태교육,0.101385,물놀이 하기 좋은,0.136606,가족,0.079642
5,힐링,0.101486,힐링,0.071213,수영장 있는,0.100898,축제,0.090279,사이트 간격이 넓은,0.079296
6,바다가 보이는,0.069342,먹거리가 풍부한,0.067489,물놀이 하기 좋은,0.087668,커플,0.037510,친절한,0.064356
7,사이트 간격이 넓은,0.025360,수영장 있는,0.044066,재미있는,0.081981,여유있는,0.022442,물놀이 하기 좋은,0.045970
8,가족,0.014687,계곡 옆,0.023911,사이트 간격이 넓은,0.027160,가족,0.017064,계곡 옆,0.043903
9,친절한,0.003546,가족,0.021071,익스트림,0.014018,바다가 보이는,0.000036,힐링,0.041128


In [333]:
cluster_lda2(3) 

/Users/hyunsil/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,0,1,0,1,0,1,0,1
0,차 대기 편한,0.490825,시설이 좋은,0.189451,문화유적,0.180187,재미있는,0.037043,별보기 좋은,0.093216
1,자전거 타기 좋은,0.251350,접근성 좋은,0.182022,생태교육,0.162108,커플,0.037042,계곡 옆,0.092904
2,시설이 좋은,0.127719,친절한,0.162527,물놀이 하기 좋은,0.133792,깨끗한,0.037041,여유있는,0.092484
3,가족,0.114915,뷰가 좋은,0.131136,재미있는,0.117362,둘레길,0.037041,그늘이 많은,0.086364
4,커플,0.000661,수영장 있는,0.112067,둘레길,0.108442,그늘이 많은,0.037040,물맑은,0.085715
5,깨끗한,0.000661,먹거리가 풍부한,0.076507,축제,0.066352,여유있는,0.037039,커플,0.079305
6,그늘이 많은,0.000661,깨끗한,0.073996,아이들 놀기 좋은,0.055319,친절한,0.037038,아이들 놀기 좋은,0.075088
7,여유있는,0.000661,힐링,0.069010,커플,0.042719,힐링,0.037038,가족,0.068971
8,친절한,0.000661,가족,0.002511,수영장 있는,0.034332,물놀이 하기 좋은,0.037037,힐링,0.064775
9,힐링,0.000661,커플,0.000043,친절한,0.033199,뷰가 좋은,0.037037,온수 잘 나오는,0.057827


In [334]:
cluster_lda2(4) 

/Users/hyunsil/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,0,1,0,1,0,1,0,1
0,여유있는,0.111259,시설이 좋은,0.159293,계곡 옆,0.225413,온수 잘 나오는,0.123547,생태교육,0.212876
1,별보기 좋은,0.103228,깨끗한,0.151198,문화유적,0.116793,아이들 놀기 좋은,0.121126,문화유적,0.179076
2,계곡 옆,0.096048,뷰가 좋은,0.147571,커플,0.111884,사이트 간격이 넓은,0.112389,물놀이 하기 좋은,0.164621
3,그늘이 많은,0.095281,접근성 좋은,0.136043,친절한,0.107065,차 대기 편한,0.100852,축제,0.125993
4,물맑은,0.075760,힐링,0.116499,가족,0.091671,커플,0.088491,둘레길,0.114850
5,커플,0.072783,친절한,0.101951,힐링,0.081201,둘레길,0.077738,수영장 있는,0.099568
6,바다가 보이는,0.069935,먹거리가 풍부한,0.090096,축제,0.076707,가족,0.070230,재미있는,0.093215
7,가족,0.068075,수영장 있는,0.088382,접근성 좋은,0.075902,재미있는,0.067289,친절한,0.007912
8,아이들 놀기 좋은,0.064304,가족,0.008717,깨끗한,0.064652,문화유적,0.064668,커플,0.000099
9,사이트 간격이 넓은,0.042731,커플,0.000014,시설이 좋은,0.047168,물놀이 하기 좋은,0.050957,별보기 좋은,0.000099


In [335]:
cluster_lda2(5) 

/Users/hyunsil/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,0,1,0,1,0,1,0,1
0,별보기 좋은,0.152329,온수 잘 나오는,0.158610,시설이 좋은,0.202401,생태교육,0.144738,시설이 좋은,0.037041
1,그늘이 많은,0.141085,차 대기 편한,0.153565,접근성 좋은,0.182546,물놀이 하기 좋은,0.126609,자전거 타기 좋은,0.037041
2,여유있는,0.121884,사이트 간격이 넓은,0.141941,뷰가 좋은,0.153875,재미있는,0.116868,아이들 놀기 좋은,0.037040
3,물맑은,0.115730,친절한,0.080213,깨끗한,0.118529,문화유적,0.113446,커플,0.037039
4,힐링,0.082824,아이들 놀기 좋은,0.076659,힐링,0.112880,둘레길,0.102531,온수 잘 나오는,0.037039
5,바다가 보이는,0.082621,커플,0.073617,친절한,0.106227,수영장 있는,0.102521,친절한,0.037039
6,계곡 옆,0.082106,가족,0.056282,먹거리가 풍부한,0.072902,축제,0.084990,문화유적,0.037039
7,커플,0.063641,물놀이 하기 좋은,0.047476,계곡 옆,0.048525,자전거 타기 좋은,0.066536,그늘이 많은,0.037038
8,아이들 놀기 좋은,0.054442,계곡 옆,0.044135,가족,0.001408,아이들 놀기 좋은,0.058436,별보기 좋은,0.037037
9,가족,0.038187,문화유적,0.039271,차 대기 편한,0.000039,커플,0.051443,계곡 옆,0.037036
